# Grupo 03 - Desafío 01
## Estrategia de construcción - Desarrollo
>**OBJETIVO:** Crear un df de post-producción que luego se transformará en producción para realizar análisis del dataset.
> * Se trabajará depurando cada columna del dataset con la finalidad de reconstruir datos perdidos y/o crear nuevas columnas que serán utilizadas a posteriori en la fase de análisis.

### Instalación de librerías

In [ ]:
#!pip install unidecode


### Importación de librerías

In [1]:
import sys
sys.path.append("../src")
import g3utils as g3

import pandas as pd
import numpy as np
import re as re
import unidecode

### Carga e inicialización de df

In [2]:
# Definición de filas a cargar en las pruebas
ca_filas = 40

In [3]:
# Carga del dataset en un df

df = pd.DataFrame(pd.read_csv("../data/properatti_minusculas.csv", encoding='UTF-8'))
#!head nombreArchivo.cvs --> comando de linux para ver las 5 primeras lineas. Se puede utilizar para ver como estan separados los datos

In [ ]:
# Para ver el contenido total de las columnas

pd.set_option('display.max_colwidth', -1)

In [4]:
# Reemplado de NaN's
df = g3.reemplaza_nan('sin datos',df)

In [5]:
df.shape

(121220, 27)

In [ ]:
# Verificación del reemplazo
df.sample(1)

In [ ]:
df.columns.unique

In [ ]:
df.columns

## Agrega de caracteristicas de data frame original a pospo
* Este bloque se reserva para realizar la precarga inicial dentro de un df de post-producción que finalmente se transformará en dataset de producción.

In [ ]:
df[(df['surface_total_in_m2'].isnull())&(df['surface_covered_in_m2'].isnull())].shape

In [ ]:
p = pospo

In [ ]:
lista_carac = ['property_type','place_name','state_name']

In [ ]:
#pospo.merge(df[lista_carac].head(ca_filas), left_index=True, right_index=True)

In [ ]:
df[(df['lat-lon'].notnull())&(df.geonames_id.isnull())].shape 

## Armado inicial de df's
* Este bloque se reserva para realizar la precarga inicial dentro de un df de post-producción que finalmente se transformará en dataset de producción.

In [6]:
# Dadtaframe de posproducción. Tras depurar todas las regex, este DF será la base de análisis de negocio
# Se ejecuta una única vez y se le van incorporando las columnas depuradas.
# Tener en cuenta que el DF pospo definitivo tiene que tener la misma cantidad de filas que el DF base len(df)

pospo = g3.generar_df_posproduccion(ca_filas)
pospo.shape
#pospo

(40, 1)

In [7]:
# Definición de df que permite registrar los valores recuperados de cada columna

df_recup = pd.DataFrame()
df_recup.shape

(0, 0)

## Procesamiento de limpieza y carga de columnas depuradas en df post-produccion 'pospo'
> *** Modo de uso:** 
>* En este markdown se vuelcan las funciones que generan y agregan columnas depuradas dentro de pospo
>* La codificación de desarrollo se realiza mas abajo dentro del título **Desarrollo** y una vez terminado se empaqueta dentro de una función consolidadora.



In [ ]:
def recuperar_usd(pospo, recup):
    pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?u[$sd]\w?|u[$sd]\w?\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio_usd
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado(df1, 'r_usd') # Genera DF acotado con los valores recuperados unicamente

    recup = g3.registrar_recupero(len(df2),'r_usd', recup)

    pospo = pd.merge(pospo, df2, on='indice', how='left')
    return pospo, recup

In [ ]:
def recuperar_metros(pospo, recup):
    pattern = r"(\d*)\,?\d*\s?(?:m²|mts\s?2|metros\s?2|mts²|m2|metros\s?cuadrado|mts\s?cuadrado)" # superficie
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'r_metros2') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_simple(df3, 'r_metros2')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_metros2', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [8]:
def recuperar_garage(pospo, recup):
    pattern = r"(garage)|(cochera)" # garage
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_garage') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_booleano(df3, 'r_garage')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_garage', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [9]:
pospo, df_recup = recuperar_garage(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (40, 2)
df_recup.size ==> (1, 1)


In [12]:
def recuperar_amenities(pospo, recup):
    pattern = r"(pileta)|(piscina)|(SUM)|(laundry)|(lavadero)|(terraza)|(solarium)|(baulera)|(sauna)|(gimnasio)|(salon de usos multiples)|(cochera)|(garage)|(gim)|(gym)" # garage
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_amenities') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = g3.obtener_df_indexado_booleano(df3, 'r_amenities')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_amenities', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')
    return pospo, recup

In [13]:
pospo, df_recup = recuperar_amenities(pospo, df_recup)
print('pospo.size ==> ' + str(pospo.shape))
print('df_recup.size ==> ' + str(df_recup.shape))

pospo.size ==> (40, 3)
df_recup.size ==> (1, 2)


In [15]:
pospo

,indice,r_garage,r_amenities
0,0,NaN,1.0
1,1,1.0,1.0
2,2,NaN,1.0
3,3,NaN,NaN
4,4,NaN,NaN
5,5,NaN,1.0
6,6,NaN,1.0
7,7,NaN,1.0
8,8,NaN,1.0
9,9,NaN,1.0


In [ ]:
r"(pileta)|(piscina)|(SUM)|(laundry)|(lavadero)|(terraza)|(solarium)|(baulera)|(sauna)|(gimnasio)|(salon de usos multiples)|(cochera)|(garage)|(gim)|(gym)"

In [ ]:
    pattern = r"(pileta)|(piscina)|(SUM)|(laundry)|(lavadero)|(terraza)|(solarium)|(baulera)|(sauna)|(gimnasio)|(salon de usos multiples)|(cochera)|(garage)|(gim)|(gym)"
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_booleano(df1, 'r_amenities') # Genera DF acotado con los valores recuperados unicamente
    
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = obtener_df_indexado_booleano(df3, 'r_amenities')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')
    
    recup = g3.registrar_recupero(len(df5),'r_amenities', recup)
    
    pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')

In [ ]:
pattern = r"(pileta)|(piscina)|(SUM)|(laundry)|(terraza)|(solarium)|(baulera)|(sauna)|(gimnasio)|(salon de usos multiples)|(gim)|(gym)"
df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
df2 = obtener_df_indexado_booleano(df1, 'r_amenities') # Genera DF acotado con los valores recuperados unicamente
 

In [ ]:
df1

In [ ]:
    df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
    df4 = obtener_df_indexado_booleano(df3, 'r_amenities')
    
    df5 = pd.concat([df2,df4]).sort_values('indice')


In [ ]:
df5

In [ ]:
display(df5, len(df5))

In [ ]:
display(df2, len(df2))

In [ ]:
pospo

In [ ]:
df.iloc[1].description

In [ ]:
def obtener_df_indexado_booleano(df, ncol, clave=None):
    """
    Busca en datos no-None y confecciona un data frame indexado incluyendo nombres a las columnas.
    Adicionalmente puede buscar por una clave distinta de None que puede ingresarse como parámetro.
    A diferencia de 'obtener_df_indexado', 'obtener_df_indexado_simple' operara sobre estructuras de datos simples como un str o int

    Parameters:
    -----------
    arg : df, ncol, clave [Opcional, default = None]
    
    df -- data frame
    ncol -- nombre de columna indexada
    clave -- parámetro opcional. Clave distinta de None. Por defecto es None
    
    Returns:
    --------
    df_ret : data frame de dos dimensiones (indice & ncol).
    
    """

    l_ind = []
    l_col = []
    for i in range(len(df.index)):
        if df.iloc[i] != clave:
            l_ind.append(i)
            l_col.append(1.00)
    df_ret = pd.DataFrame(l_ind, columns={'indice'})
    df_ret[ncol] = pd.DataFrame(l_col)
    return df_ret


In [ ]:
key = 'contrafrente'
desc = g3.existe_clave(key, 'description', df)
tit = g3.existe_clave(key, 'title', df)
print('Datos recuperados: "description" ==> ',desc, '"title" ==> ',tit, '"Total" ==> ',desc+tit)


In [ ]:
#pospo

In [ ]:
g3.crear_csv('test.csv',df_recup)

In [ ]:
df_recup

# Desarrollo

In [ ]:
def busca_claves(pattern, columna, df_aux):
    regex = re.compile(pattern, flags = re.IGNORECASE)
    m = pd.DataFrame([regex.findall(n) for n in df_aux[columna]])
    return m[0]

In [ ]:
def registrar_recupero(valor, colu, df_recup):
    
    if df_recup.size == 0:     
        df_recup = pd.DataFrame([valor], columns=[colu])
        print('primer dato en df')
    elif colu not in df_recup.columns:
        df_recup[colu] = [valor+1]
        print('columna creada')
    else:
        df_recup[colu] = [valor+2]
        print('columna existente')
    return df_recup
    


In [ ]:
def crear_csv(nombre, def_recup):
    ruta = '../data/'+nombre
    df_recup.to_csv(ruta, encoding='utf-8')
    return print('[LOG] Se ha creado el archivo: ',nombre)


In [ ]:
len(df1),len(df2)

In [ ]:
df3 = g3.busca_claves(pattern,'title',df.head(ca_filas))
df4 = g3.obtener_df_indexado_simple(df3, 'r_frente')
    


In [ ]:
len(df3),len(df4)

In [ ]:
df5 = pd.concat([df2,df4]).sort_values('indice')
    
pospo = pd.merge(pospo, df5,on='indice', how='left').drop_duplicates(['indice'],keep='last')

In [ ]:
len(df5),len(pospo)

In [ ]:
display(df2, df4)


### Patterns

In [ ]:
# Comentar lo que no se deba ejecutar
# Listado de todos los paterns configurados
# regex = re.compile(pattern, flags = re.IGNORECASE) # Se ejecuta dentro de funciones específicas

# pattern = r'(\d+)\s[m]2' # Patron para buscar metros cuadrados (m2)
# pattern = r'(\d*\w*)\s*amb'  # Patron para buscar ambientes OK
# pattern = r'(\d*)\s?\b[aA]mbientes\w*'
# pattern = r'(\d*\w*)\s*usd'
# pattern = r'(\d*\w*)\s*habitacion|(\d*\w*)\s?cuarto|(\d*\w*)\s?dorm|(\d*\w*)\s?pieza'
# pattern = r"(pileta)|(piscina)|(SUM)|(laundry)|(lavadero)|(terraza)|(solarium)|(baulera)|(sauna)|(gimnasio)|(salon de usos multiples)|(cochera)|(garage)|(gim)|(gym)" # amenities
# pattern = r"(triplex)|(duplex)|(frente)|(contrafrente)|(PH)|(chalet)" # casa
# pattern = r"(casa)|(departamento)|(triplex)|(duplex)|(frente)|(contrafrente)|(PH)|(chalet)" # casa
# pattern = r"(\d*)\,?\d*\s?(?:m²|mts\s?2|metros\s?2|mts²|m2|metros\s?cuadrado|mts\s?cuadrado)" # superficie
# pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?u[$sd]\w?|u[$sd]\w?\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio_usd OK
# pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?\$|\$\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio
pattern = r"(garage)|(cochera)" # garage



### Regex

> Depuración sobre feature **'description'**

In [ ]:
key = 'contrafrente'
desc = existe_clave(key, 'description', df)
tit = existe_clave(key, 'title', df)
print('Datos recuperados: "description" ==> ',desc, '"title" ==> ',tit)
print('Total recuperados ==> ',desc+tit)

In [ ]:
pospo

In [ ]:
pattern = r"(frente)|(contrafrente)" # casa
df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado

#df1

In [ ]:
df1 = g3.reemplaza_nan('sin datos',df1)
df1[0]=='sin dato'

In [ ]:
len(df1[2])

In [ ]:
# Ejecuta función de busqueda de claves en base a los Patterns previamente configurados

df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
#df1 = g3.busca_claves(pattern,'description',df) # Para trabajar con el DF completo
#df1 = g3.reemplaza_nan('sin datos',df1)
df1
#df1.iloc[2]
#len(df1.iloc[2])

In [ ]:
df2 = g3.obtener_df_indexado(df1, 'metros2') # Genera DF acotado con los valores recuperados unicamente
df2

In [ ]:
len(df)

### [FIN] Procesamiento de limpieza y carga de columna depurada en df post-produccion

### Procesamiento de armado intermedio de df de post-produccion
> * Modo de uso temporal hasta terminar de depurar todas las claves necesarias
> * Cada grupo de código se utiliza para realizar consolidado de filas dentro del DF que luego será el definitivo

### Armado de dataset definitivo

In [ ]:
# Agregado de columna limpia a DF definitivo
dfx = g3.agregar_columna('ambiente', dfx, df_temp)

# ------------------------------
# SIN USO
# ------------------------------

In [ ]:
#tmp_df = df["cases"].str.extract("([a-z])([0-9]+)([0-9]{2})",expand=True) 
tmp_df = df["description"].str.extract("((\d*)\s?\b[aA]mb\w*)",expand=True) 
tmp_df.dropna(axis=0)

In [ ]:
# TEST: Regex de prueba. Alimenta un DF con el resultado
# CONDICION: No lee NaN's
# NOTA: Me di cuenta que REGEX analiza todo el str y en muchos casos detecta la KEY en varias partes del 
#   str, ergo, al capturar el siguiente dato, lo incluye dentro de una lista y eso lo representa en otra columna
#   Entonces ponerle nombre a una única columna no tendría sentido. Si verificamos con .head(10) no vamos
#   a tener problemas porque justo las 10 primeras filas no tienen mas de una KEY en el str a analizar...

#m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(10)], columns=["ambientes"] )
m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(50)])
#m = pd.DataFrame([regex.findall(n) for n in df_a1['description']])
display(len(m))
#m
m[0]


### Pruebas de campos

In [ ]:
# Para verificar el contenido de alguna fila al azar
df.sample(1)['description']

In [ ]:
# Para verificar algún problema en alguna fila en particular
df.iloc[280]['description']

In [ ]:
#df.iloc[277]

### Reemplazo de NaN's
Esto se realiza porque la función de búsqueda explotaba 

In [ ]:
df_a1=df.replace(np.nan,'sin-dato')
#df_a1.head(2)

In [ ]:
def reemplaza_nan(df, clave):
    return df.replace(np.nan,clave)

### Funciones

In [ ]:
# Busca claves segun pattern en columna dentro de dataFrame 
# Entrada: pattern, columna, df
# Salida: DataFrame

def busca_claves(pattern, columna, df_aux):
    regex = re.compile(pattern, flags = re.IGNORECASE)
    m = pd.DataFrame([regex.findall(n) for n in df_aux[columna].head(10000)])
    return m[0]

In [ ]:
def existe_clave(key, columna, df_aux):    
    m = []
    for frase in df_aux[columna]:
        if key in frase:
            m.append(True)
        else:
            m.append(False)
    return sum(m)


In [ ]:
# No se está utilizando mas desde que el dataset se encuentra en minúsculas y sin acentos
def quitar_caracteres(column):
    for i in range(len(column)):
        if type(column[i]) == str:
            column[i] = str.lower(unidecode.unidecode(column[i]))

In [ ]:
# Cambia los literales numericos x floats. En caso de no detectar la clave en un diccionario, reemplaza por NaN.
# El reemplazo x NaN se hace para poder realizar operaciones.
# Mejoras: debiera recibir diccionario y lista_prop como parametros para dejar una función polimorfica. 
# Entrada: DataFrame
# Salida: DataFrame

def cambiar_x_nros(df):
    dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
    lista_prop = ['mono','un','uno','dos','tres','cuatro','cinco','seis','siete']
    serie_1 = pd.Series([x if x not in lista_prop else dic.get(x) for x in df])
    serie_2 = pd.to_numeric(serie_1, errors='coerse', downcast='float') # coerse: pasa a NaN los no-numericos
    return pd.DataFrame(serie_2)

In [ ]:
quitar_caracteres(df["place_name"])
quitar_caracteres(df["place_with_parent_names"])
quitar_caracteres(df["country_name"])
quitar_caracteres(df["state_name"])
quitar_caracteres(df["description"])
quitar_caracteres(df["title"])
